In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, GRU

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from sklearn.metrics import f1_score

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         


In [7]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test.csv.zip

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                


In [8]:
!unzip /content/drive/My\ Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/train.csv.zip 

Archive:  /content/drive/My Drive/Kaggle/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               


In [9]:
!ls /content/drive/MyDrive/Kaggle/jigsaw-toxic-comment-classification-challenge

sample_submission.csv.zip  test.csv.zip  test_labels.csv.zip  train.csv.zip


In [10]:
!realpath test.csv

/content/test.csv


In [11]:
training_data = pd.read_csv("/content/train.csv")
testing_data = pd.read_csv("/content/test.csv")
testing_labels = pd.read_csv("/content/test_labels.csv")

In [12]:
drive.flush_and_unmount()

In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-06-15 07:30:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-15 07:30:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-15 07:30:47--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [14]:
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip  test.csv	     train.csv
glove.6B.200d.txt  glove.6B.50d.txt   sample_data   test_labels.csv


In [15]:
embedding = "glove.6B.50d.txt"

In [16]:
# size of embedding vector (total nb of words)
max_len_embedding = 20000
# size of each word
size = 50
# maximum length of a comment (in words)
max_len = 100

In [17]:
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
nb_labels = len(labels)

In [18]:
# Preprocessing

# Pattern form filtering english stopwords, taken from https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python
stopword_pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

def preprocess_data(text):

  # Removing trailing spaces and leading spaces
  text = text.strip()
  # Lowercase
  text = text.lower()
  # Remove numbers
  text = re.sub(r'[0-9]', '', text)
  # Remove stopwords
  text = stopword_pattern.sub('', text)
  # Remove special characters
  text = re.sub(r'[^\w\s]', '', text)
  # Remove consecutive spaces
  text = re.sub(r' +', ' ', text)
  # Remove new lines
  text = re.sub(r'\n', ' ', text)

  return text

In [19]:
training_data.comment_text = training_data["comment_text"].map(preprocess_data)

In [20]:
train_y = training_data[labels].fillna("_na_").values
testing_data.comment_text = testing_data["comment_text"].map(preprocess_data)

In [21]:
testing_clean = pd.merge(testing_data, testing_labels)
testing_clean = testing_clean.drop(testing_clean.index[testing_clean['toxic'] == -1])
test_y = testing_clean[labels].fillna("_na_").values
train_list = training_data['comment_text'].tolist()
test_list = testing_clean['comment_text'].tolist()
len(test_list)

63978

In [22]:
tokenizer = Tokenizer(num_words=max_len_embedding)

In [23]:
tokenizer = Tokenizer(num_words=max_len_embedding)
tokenizer.fit_on_texts(train_list)
list_tokenized_train = tokenizer.texts_to_sequences(train_list)
list_tokenized_test = tokenizer.texts_to_sequences(test_list)

train_x = pad_sequences(list_tokenized_train, maxlen=max_len)
test_x = pad_sequences(list_tokenized_test, maxlen=max_len)

In [24]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

embeddings_dict = dict(get_coefs(*elt.strip().split()) for elt in open(embedding))

In [25]:
embedding_stack = np.stack(embeddings_dict.values())

mean = embedding_stack.mean()
std = embedding_stack.std()
print(mean)
std

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


0.020940498


0.6441043

In [26]:
index = tokenizer.word_index
total_words = min(max_len_embedding, len(index))
embedding_matrix = np.random.normal(mean, std, (total_words, size))
for word, i in index.items():
    if i >= max_len_embedding: 
      continue
    embedding_vector = index.get(word)
    if embedding_vector is not None: 
      embedding_matrix[i] = embedding_vector

In [27]:
lstm_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[embedding_matrix]),
  tf.keras.layers.Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

lstm_model.summary()

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 100)         40400     
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3

In [28]:
lstm_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.1);

Epoch 1/2
4488/4488 [==============================] - 3339s 742ms/step - loss: 0.1433 - accuracy: 0.9105 - val_loss: 0.1400 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 3322s 740ms/step - loss: 0.1391 - accuracy: 0.9928 - val_loss: 0.1404 - val_accuracy: 0.9940


In [62]:
y_test_lstm = lstm_model.predict([test_x], batch_size=1024, verbose=1) 
y_test_lstm

63/63 [==============================] - 4s 61ms/step


array([[0.11432589, 0.01262311, 0.06208618, 0.00296796, 0.06028283,
        0.00849418],
       [0.12836766, 0.01573337, 0.07240162, 0.00359868, 0.06993145,
        0.01013626],
       [0.0688803 , 0.00488057, 0.03219996, 0.00125635, 0.03127804,
        0.00397293],
       ...,
       [0.12479429, 0.01491869, 0.06974557, 0.00343779, 0.06745971,
        0.00971683],
       [0.09921107, 0.00973108, 0.05154187, 0.00237311, 0.05038292,
        0.00691643],
       [0.09636432, 0.00922339, 0.04961004, 0.00226521, 0.04855976,
        0.00662752]], dtype=float32)

In [31]:
f1_score_val_lstm = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_lstm, axis=1), average="weighted")
f1_score_val_lstm

0.9964142608765079

In [200]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_lstm[:, i] >= 0.1
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.6666223929095505
severe_toxic :
0.9914037297776611
obscene :
0.9143192385431218
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.9118546812692627


In [33]:
# export results

results = testing_clean.copy(deep=True)
results[labels] = y_test_lstm

results.to_csv("lstm_predictions.csv")

In [34]:
!realpath lstm_predictions.csv

/content/lstm_predictions.csv


In [35]:
# GRU model

In [139]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 


In [45]:
gru_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[embedding_matrix]),
  tf.keras.layers.SpatialDropout1D(0.3),
  tf.keras.layers.Bidirectional(GRU(70, return_sequences=True)),  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

gru_model.summary()

# gru_model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(
    # gamma=4.0,
    # from_logits=False,
    # label_smoothing=0.0,
    # axis=-1,
    # reduction=tf.keras.losses.Reduction.SUM,
#     name='binary_focal_crossentropy'
#     ), optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC()])
# import tensorflow_addons as tfa
# gru_model.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(
#     gamma=nb_labels,
#     from_logits=False,
#     reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE,
# ), optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC()])

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC()])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 50)           1000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 50)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 140)         51240     
 nal)                                                            
                                                                 
 global_average_pooling1d_1   (None, 140)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 50)                7050      
                                                      

In [46]:
gru_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.5)

Epoch 1/2
2494/2494 [==============================] - 53s 20ms/step - loss: 0.1458 - accuracy: 0.8500 - auc: 0.7548 - val_loss: 0.1359 - val_accuracy: 0.9940 - val_auc: 0.7917
Epoch 2/2
2494/2494 [==============================] - 45s 18ms/step - loss: 0.1399 - accuracy: 0.9830 - auc: 0.7789 - val_loss: 0.1345 - val_accuracy: 0.9940 - val_auc: 0.7983


In [47]:
y_test_gru = gru_model.predict([test_x], batch_size=1024, verbose=1)
y_test_gru

63/63 [==============================] - 2s 19ms/step


array([[0.11065701, 0.01192847, 0.06048493, 0.00281989, 0.05888237,
        0.01000555],
       [0.14161117, 0.01972355, 0.08396191, 0.00474785, 0.08038125,
        0.01566022],
       [0.04477889, 0.00243768, 0.0198296 , 0.00056377, 0.01922448,
        0.00228969],
       ...,
       [0.13987288, 0.01923357, 0.08275157, 0.00466386, 0.07916053,
        0.01544101],
       [0.12654282, 0.01570627, 0.07356481, 0.00403724, 0.06995023,
        0.01377526],
       [0.12494688, 0.01531075, 0.07247613, 0.00396425, 0.06886562,
        0.01357753]], dtype=float32)

In [48]:
gru_model.evaluate(test_x, test_y, batch_size=1024, verbose=1)

63/63 [==============================] - 1s 19ms/step - loss: 0.1373 - accuracy: 0.9976 - auc: 0.8062


[0.1372605413198471, 0.9976085424423218, 0.8062136173248291]

In [50]:
f1_score_val_gru = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_gru, axis=1), average="weighted")
f1_score_val_gru

0.9964142608765079

In [52]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_gru[:, i] >= 0.1
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.5881450870154555
severe_toxic :
0.9914037297776611
obscene :
0.9145761654940144
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.8988179514453957


In [41]:
# GRU model is faster to train but doesn't perform as good for "toxic" comments

In [54]:
results = testing_clean.copy(deep=True)
results[labels] = y_test_gru

results.to_csv("gru_predictions.csv")

In [42]:
# subword embedding (fasttext) : trying to improve toxic obscene and insult labels f1 scores

In [27]:
fasttext_embedding_file = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!unzip /content/drive/MyDrive/Kaggle/crawl-300d-2M.vec.zip

Archive:  /content/drive/MyDrive/Kaggle/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       
  inflating: __MACOSX/._crawl-300d-2M.vec  


In [30]:
!realpath crawl-300d-2M.vec

/content/crawl-300d-2M.vec


In [31]:
fasttext_embedding_file = "/content/crawl-300d-2M.vec"

In [32]:
fasttext_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(fasttext_embedding_file))

In [33]:
drive.flush_and_unmount()

In [34]:
fasttext_stack = np.stack(fasttext_index.values())

mean = fasttext_stack.mean()
std = fasttext_stack.std()
print(mean)
std

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


0.0055286596


0.34703913

In [35]:
index = tokenizer.word_index
total_words = min(max_len_embedding, len(index))
fasttext_embedding_matrix = np.random.normal(mean, std, (total_words, size))
for word, i in index.items():
    if i >= max_len_embedding: continue
    embedding_vector = index.get(word)
    if embedding_vector is not None: fasttext_embedding_matrix[i] = embedding_vector

In [52]:
# Reimplementing the model with the better accuracy using subword embeddings (bidirectional GRU and LSTM models implemented have the same accuracy, so I chose GRU because it was faster during training)

In [36]:
gru_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(max_len,)),
  tf.keras.layers.Embedding(input_dim=max_len_embedding, output_dim=size, input_length=max_len_embedding, weights=[fasttext_embedding_matrix]),
  tf.keras.layers.SpatialDropout1D(0.3),
  tf.keras.layers.Bidirectional(GRU(70, return_sequences=True)),  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(50, activation="relu"),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(nb_labels, activation="sigmoid")
])

In [37]:
gru_model.summary()

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 50)          0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 100, 140)         51240     
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 140)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 50)                7050      
                                                        

In [38]:
gru_model.fit(train_x, train_y, batch_size=32, epochs=2, validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 65s 12ms/step - loss: 0.1412 - accuracy: 0.9312 - val_loss: 0.1411 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 54s 12ms/step - loss: 0.1370 - accuracy: 0.9936 - val_loss: 0.1385 - val_accuracy: 0.9940


In [39]:
y_test_gru_ft = gru_model.predict([test_x], batch_size=1024, verbose=1)
y_test_gru_ft

63/63 [==============================] - 1s 14ms/step


array([[0.05896901, 0.00280833, 0.03040114, 0.00048335, 0.02597806,
        0.00262513],
       [0.11278635, 0.01012663, 0.06459665, 0.00213205, 0.05976514,
        0.00824218],
       [0.03038359, 0.00088624, 0.01341038, 0.0001424 , 0.0110142 ,
        0.00110165],
       ...,
       [0.11184305, 0.01002471, 0.06387516, 0.0021197 , 0.0590764 ,
        0.00823427],
       [0.10459071, 0.00925401, 0.05840017, 0.00202596, 0.05386159,
        0.00817895],
       [0.10371307, 0.00916172, 0.057747  , 0.00201448, 0.05324051,
        0.00817185]], dtype=float32)

In [40]:
f1_score_val_gru_ft = f1_score(np.argmax(testing_clean[labels].values, axis=1), np.argmax(y_test_gru_ft, axis=1), average="weighted")
f1_score_val_gru_ft

0.9964142608765079

In [44]:
avg = 0
for i, label in enumerate(labels):
    print(label, ":")
    pb = y_test_gru_ft[:, i] >= 0.1
    score = f1_score(testing_clean[label], pb, average="weighted")
    print(score)
    avg += score

avg /= nb_labels
print("Average f1-score:", avg)

toxic :
0.7024372340635118
severe_toxic :
0.9914037297776611
obscene :
0.9143192385431218
threat :
0.995055710671004
insult :
0.9203891180722282
identity_hate :
0.9833378976420114
Average f1-score: 0.9178238214615897


In [59]:
# fasttext performs better

In [53]:
results = testing_clean.copy(deep=True)
results[labels] = y_test_gru_ft

results.to_csv("gru_ft_predictions.csv")

In [43]:
!pip install casanova

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Adjusting results for ensemble model

In [71]:
import casanova

with open("gru_ft_predictions.csv") as f, \
     open("gru_ft_adjusted_preds.csv", "w") as of:
  enricher = casanova.enricher(f, of)

  for row in enricher:
    toxic_val = float(row[3])
    row[3] = str(toxic_val / 0.2)

    severe_toxic_val = float(row[4])
    row[4] = str(severe_toxic_val / 0.2)

    obscene_val = float(row[5])
    row[5] = str(obscene_val / 0.2)

    threat_val = float(row[6])
    row[6] = str(threat_val / 0.2)

    insult_val = float(row[7])
    row[7] = str(insult_val / 0.2)

    identity_hate_val = float(row[8])
    row[8] = str(identity_hate_val / 0.2)

    enricher.writerow(row)

In [76]:
with open("gru_predictions.csv") as f, \
     open("gru_adjusted_preds.csv", "w") as of:
  enricher = casanova.enricher(f, of)
  toxic_pos = enricher.headers.toxic
  severe_toxic_pos = enricher.headers.severe_toxic
  obscene_pos = enricher.headers.obscene
  threat_pos = enricher.headers.threat
  insult_pos = enricher.headers.insult
  identity_hate_pos = enricher.headers.identity_hate

  for row in enricher:
    toxic_val = float(row[toxic_pos])
    row[toxic_pos] = str(toxic_val / 0.2)

    severe_toxic_val = float(row[severe_toxic_pos])
    row[severe_toxic_pos] = str(severe_toxic_val / 0.2)

    obscene_val = float(row[obscene_pos])
    row[obscene_pos] = str(obscene_val / 0.2)

    threat_val = float(row[threat_pos])
    row[threat_pos] = str(threat_val / 0.2)

    insult_val = float(row[insult_pos])
    row[insult_pos] = str(insult_val / 0.2)

    identity_hate_val = float(row[identity_hate_pos])
    row[identity_hate_pos] = str(identity_hate_val / 0.2)

    enricher.writerow(row)

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
with open("/content/drive/MyDrive/Kaggle/predictions/lstm_predictions.csv") as f, \
     open("lstm_adjusted_preds.csv", "w") as of:
  enricher = casanova.enricher(f, of)
  toxic_pos = enricher.headers.toxic
  severe_toxic_pos = enricher.headers.severe_toxic
  obscene_pos = enricher.headers.obscene
  threat_pos = enricher.headers.threat
  insult_pos = enricher.headers.insult
  identity_hate_pos = enricher.headers.identity_hate

  for row in enricher:
    toxic_val = float(row[toxic_pos])
    row[toxic_pos] = str(toxic_val / 0.2)

    severe_toxic_val = float(row[severe_toxic_pos])
    row[severe_toxic_pos] = str(severe_toxic_val / 0.2)

    obscene_val = float(row[obscene_pos])
    row[obscene_pos] = str(obscene_val / 0.2)

    threat_val = float(row[threat_pos])
    row[threat_pos] = str(threat_val / 0.2)

    insult_val = float(row[insult_pos])
    row[insult_pos] = str(insult_val / 0.2)

    identity_hate_val = float(row[identity_hate_pos])
    row[identity_hate_pos] = str(identity_hate_val / 0.2)

    enricher.writerow(row)

In [75]:
drive.flush_and_unmount()